In [ ]:
import torch
import torchvision
torch.__version__

## 4.5 多GPU并行训练
在我们进行神经网络训练的时候，因为计算量巨大所以单个GPU运算会使得计算时间很长，使得我们不能够及时的得到结果，例如我们如果使用但GPU使用ImageNet的数据训练一个分类器，可能会花费一周甚至一个月的时间。所以在Pytorch中引入了多GPU计算的机制，这样使得训练速度可以指数级的增长。

### `torch.nn.DataParalle`
一般情况下我们都会使用一台主机带多个显卡，这样是一个最节省预算的方案，在Pytorch中为我们提供了一个非常简单的方法来支持但主机多GPU，那就`torch.nn.DataParalle`我们只要将我们自己的模型作为参数，直接传入即可

In [ ]:
#使用内置的一个模型，我们这里以resnet50为例
model = torchvision.models.resnet50()

In [ ]:
# 模型使用多GPU
mdp = torch.nn.DataParallel(model)
mdp

只要这样一个简单的包裹，Pytorch已经为我们做了很多复杂的工作。我们只需要增大我们训练的`batch_size`(一般计算为N倍，N为显卡数量)，其他代码不需要任何改动。 虽然代码不需要做更改，但是batch size太大了训练收敛会很慢，所以还要把学习率调大一点。大学率也会使得模型的训练在早期的阶段变得十分不稳定，所以这里需要一个学习率的热身（warm up）来稳定梯度的下降，然后再逐步的提高学习率。这种热身只有在超级大的批次下才需要进行，一般我们这种一机4卡或者说在batch size 小于 5000基本上是不需要的。

DataParallel的并行处理机制是，首先将模型加载到主 GPU 上(默认的第一个GPU，GPU0为主GPU)，然后再将模型复制到各个指定的从 GPU 中，然后将输入数据按 batch 维度进行划分，具体来说就是每个 GPU 分配到的数据 batch 数量是总输入数据的 batch 除以指定 GPU 个数。每个 GPU 将针对各自的输入数据独立进行 forward 计算，最后将各个 GPU 的 loss 进行求和，再用反向传播更新单个 GPU 上的模型参数，再将更新后的模型参数复制到剩余指定的 GPU 中，这样就完成了一次迭代计算。

DataParallel其实也是一个nn.Model所以我们可以保存权重的方法和一般的nn.Model没有区别，只不过如果想使用单卡或者cpu作为推理的时候需要从里面读出原始的model。

In [ ]:
#获取到原始的model
m = mdp.module
m

DataParallel会将定义的网络模型参数默认放在GPU 0上，所以dataparallel实质是可以看做把训练参数从GPU拷贝到其他的GPU同时训练，这样会导致内存和GPU使用率出现很严重的负载不均衡现象，即GPU 0的使用内存和使用率会大大超出其他显卡的使用内存，因为在这里GPU0作为master来进行梯度的汇总和模型的更新，再将计算任务下发给其他GPU，所以他的内存和使用率会比其他的高。

所以我们使用新的`torch.distributed`来构建更为同步的分布式运算。使用`torch.distributed`不仅可以支持单机还可以支持多个主机，多个GPU进行计算。

### `torch.distributed`

`torch.distributed`相对于`torch.nn.DataParalle`是一个底层的API，所以我们要修改我们的代码，使其能够独立的在机器（节点）中运行。我们想要完全实现分布式，并且在每个结点的每个GPU上独立运行进程，这一共需要N个进程。N是我们的GPU总数，这里我们以4来计算。

首先 初始化分布式后端，封装模型以及准备数据，这些数据用于在独立的数据子集中训练进程。修改后的代码如下



In [ ]:
# 以下脚本在jupyter notebook执行肯定会不成功，请保存成py文件后测试
import torch
import argparse
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler

# 这里的node_rank是本地GPU的标识
parser = argparse.ArgumentParser()
parser.add_argument("--local_rank", type=int)
args = parser.parse_args()

# 使用Nvdea的nccl来初始化节点 
torch.distributed.init_process_group(backend ='nccl')

# 封装分配给当前进程的GPU上的模型
device = torch.device('cuda', args.local_rank)
model = model.to(device)
dustrib_model = torch.nn.parallel.DistributedDataParallel(model, device_ids = [args.node_rank], output_device = args.node_rank)

# 将数据加载限制为数据集的子集（不包括当前进程）
sampler = DistributedSampler(dataset)

dataloader = DataLoader(dataset, sampler = sampler)

for inputs, labels in dataloader:
    predictions = dustrib_model(inputs.to(device))
    loss = loss_function(predictions, labels.to(device))
    loss.backward()
    optimizer.step()

在运行时我们也不能简单的使用python文件名来执行了，我们这里需要使用PyTorch中为我们准备好的`torch.distributed.launch`运行脚本。它能自动进行环境变量的设置，并使用正确的`node_rank`参数调用脚本。

这里我们要准备一台机器作为master，所有的机器都要求能对它进行访问。因此，它需要拥有一个可以访问的IP地址（示例中为：196.168.100.100）以及一个开放的端口（示例中为：6666）。我们将使用`torch.distributed.launch`在第一台机器上运行脚本：

```
python -m torch.distributed.launch --nproc_per_node=2 --nnodes=2 --node_rank=0 --master_addr="192.168.100.100" --master_port=6666 文件名 (--arg1 --arg2 等其他参数)
```

第二台主机上只需要更改`--node_rank = 0`即可

很有可能在运行的时候报错，那是因为我们没有设置NCCL socket网络接口 我们以网卡名为ens3为例，输入
```
export NCCL_SOCKET_IFNAME=ens3
```
ens3这个名称 可以使用ifconfig命令查看确认

参数说明：
- `nproc_per_node`: 主机中包含的GPU总数
- `nnodes`: 总计的主机数
- `node_rank`: 主机中的GPU标识


`torch.distributed`不仅支持nccl还支持其他的两个后端 gloo和mpi

### `torch.utils.checkpoint`
在我们训练时，可能会遇到训练集的单个样本比内存还要大根本载入不了，那我们如何来训练呢？

pytorch为我们提供了梯度检查点（gradient-checkpointing）节省计算资源，梯度检查点会将我们连续计算的元正向和元反向传播切分成片段。但由于需要增加额外的计算以减少内存需求，该方法效率会有一些下降，但是它在某些示例中有较为明显的优势，比如在长序列上训练RNN模型。